**Connecting Python with MySQL**

In [1]:
# install necessary libraries
! pip install sqlalchemy # ==1.4.46
! pip install PyMySQL

In [7]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)

········


**Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:**

**Data for May**

In [8]:
query = '''select * from sakila.rentals_may'''

dfmay = pd.read_sql_query(query, engine)
dfmay

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


**Check the number of rentals for each customer for May**

In [22]:
query = '''select customer_id, count(rental_id) as number_of_rentals from sakila.rentals_may
group by customer_id
order by customer_id'''

df1may = pd.read_sql_query(query, engine)
df1may

,customer_id,number_of_rentals
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


**Data for June**

In [9]:
query = '''select * from sakila.rentals_june'''

dfjune = pd.read_sql_query(query, engine)
dfjune

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
2306,3465,2005-06-21 22:10:01,1488,510,2005-06-30 21:35:01,1,2006-02-15 21:30:53
2307,3466,2005-06-21 22:13:33,371,226,2005-06-25 21:01:33,2,2006-02-15 21:30:53
2308,3467,2005-06-21 22:19:25,729,543,2005-06-27 00:03:25,2,2006-02-15 21:30:53
2309,3468,2005-06-21 22:43:45,2899,100,2005-06-30 01:49:45,1,2006-02-15 21:30:53


**Check the number of rentals for each customer for June**

In [23]:
query = '''select customer_id, count(rental_id) as number_of_rentals from sakila.rentals_june
group by customer_id
order by customer_id'''

df1june = pd.read_sql_query(query, engine)
df1june

,customer_id,number_of_rentals
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


**Write a function that checks if customer borrowed more or less films in the month of June as compared to May.**

In [24]:
merged_df = pd.merge(df1may, df1june, on='customer_id', how='inner', suffixes=('_may','_june'))
merged_df

,customer_id,number_of_rentals_may,number_of_rentals_june
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4
...,...,...,...
507,594,4,6
508,595,1,2
509,596,6,2
510,597,2,3


In [28]:
def more_rentals(row):
    if row['number_of_rentals_may'] > row['number_of_rentals_june']:
        return 'rented less'
    elif row['number_of_rentals_may'] < row['number_of_rentals_june']:
        return 'rented more'
    else:
        return 'equal'

merged_df['rented_more_or_less'] = merged_df.apply(more_rentals,axis=1)

print(merged_df[['customer_id', 'rented_more_or_less']])

     customer_id rented_more_or_less
0              1         rented more
1              2               equal
2              3         rented more
3              5         rented more
4              6         rented more
..           ...                 ...
507          594         rented more
508          595         rented more
509          596         rented less
510          597         rented more
511          599         rented more

[512 rows x 2 columns]


In [29]:
def more_rentals(row):
    if row['number_of_rentals_may'] > row['number_of_rentals_june']:
        return 'may'
    elif row['number_of_rentals_may'] < row['number_of_rentals_june']:
        return 'june'
    else:
        return 'equal'

merged_df['month_with_more_rentals'] = merged_df.apply(more_rentals,axis=1)

print(merged_df[['customer_id', 'month_with_more_rentals']])

     customer_id month_with_more_rentals
0              1                    june
1              2                   equal
2              3                    june
3              5                    june
4              6                    june
..           ...                     ...
507          594                    june
508          595                    june
509          596                     may
510          597                    june
511          599                    june

[512 rows x 2 columns]
